In [2]:
import numpy as np
import pandas as pd

In [3]:
# parameters
r = 0.05
sig = 0.25
K = 101
S0 = 100
T = 0.5

BSM pricer

In [3]:
import numpy as np
from scipy.stats import norm
import scipy.stats as ss

N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

BS_CALL(S0, K, T, r, sig)

NameError: name 'S0' is not defined

European binomial tree

In [24]:
N = 1000  # number of periods or number of time steps
payoff = "put"  # payoff

dT = float(T) / N  # Delta t
u = np.exp(sig * np.sqrt(dT))  # up factor
d = 1.0 / u  # down factor

V = np.zeros(N + 1)  # initialize the price vector

# price S_T at time T
S_T = np.array([(S0 * u**j * d ** (N - j)) for j in range(N + 1)])

a = np.exp(r * dT)  # risk free compounded return
p = (a - d) / (u - d)  # risk neutral up probability
q = 1.0 - p  # risk neutral down probability

if payoff == "call":
    V[:] = np.maximum(S_T - K, 0.0)
else:
    V[:] = np.maximum(K - S_T, 0.0)

for i in range(N - 1, -1, -1):
    # the price vector is overwritten at each step
    V[:-1] = np.exp(-r * dT) * (p * V[1:] + q * V[:-1])

print("BS Tree Price: ", V[0])

BS Tree Price:  6.268569012465585


American binomial tree pricer

In [26]:
N = 1000  # number of periods or number of time steps
payoff = "call"  # payoff

dT = float(T) / N  # Delta t
u = np.exp(sig * np.sqrt(dT))  # up factor
d = 1.0 / u  # down factor

V = np.zeros(N + 1)  # initialize the price vector
S_T = np.array([(S0 * u**j * d ** (N - j)) for j in range(N + 1)])  # price S_T at time T

a = np.exp(r * dT)  # risk free compound return
p = (a - d) / (u - d)  # risk neutral up probability
q = 1.0 - p  # risk neutral down probability

if payoff == "call":
    V[:] = np.maximum(S_T - K, 0.0)
elif payoff == "put":
    V[:] = np.maximum(K - S_T, 0.0)

for i in range(N - 1, -1, -1):
    V[:-1] = np.exp(-r * dT) * (p * V[1:] + q * V[:-1])  # the price vector is overwritten at each step
    S_T = S_T * u  # it is a tricky way to obtain the price at the previous time step
    if payoff == "call":
        V = np.maximum(V, S_T - K)
    elif payoff == "put":
        V = np.maximum(V, K - S_T)

print("American BS Tree Price: ", V[0])

American BS Tree Price:  7.762267897606796


European MC pricer

In [44]:
N = 10000  # N = 10k

W = ss.norm.rvs((r - 0.5 * sig**2) * T, np.sqrt(T) * sig, N)
S_T = S0 * np.exp(W)

c_path = np.exp(-r * T) * np.maximum(S_T - K, 0)
p_path = np.exp(-r * T) * np.maximum(K - S_T, 0)

call = np.mean(c_path)
put = np.mean(p_path)

call_stderr = np.std(c_path) / np.sqrt(N)
put_stderr = np.std(p_path) / np.sqrt(N)


call_err = ss.sem(np.exp(-r * T) * np.maximum(S_T - K, 0))  # standard error
put_err = ss.sem(np.exp(-r * T) * np.maximum(K - S_T, 0))  # standard error

# my std error


In [45]:
print("Call price: {}, with error: {}".format(call, call_err))
print("Put price: {}, with error: {}".format(put, put_err))

print(call_stderr)
print(put_stderr)

Call price: 7.7565930595476935, with error: 0.11892972458396267
Put price: 6.138500390419326, with error: 0.08659887231934021
0.11892377794906389
0.08659454226747025


Two Correlated Random Processes

In [6]:
N = 100000  # N = 10k
rho = 0.8
sig1 = 0.3
sig2 = 0.25
r = 0
T = 1
paths=1000
dt = T/paths
K = 10

W1 = ss.norm.rvs((r - 0.5 * sig1**2) * dt, np.sqrt(T) * sig1, (paths, N))
W2 = ss.norm.rvs((r - 0.5 * sig2**2) * dt, np.sqrt(T) * sig2, (paths, N))

eta1 = W1 
eta2 = rho * W1 + W2 * np.sqrt(1 - rho**2)

S1_T = 50 * np.exp(eta1)
S2_T = 40 * np.exp(eta2)

path = np.exp(-r * T) * np.maximum(S1_T - S2_T - K, 0)
print(np.mean(path))
stderr = np.std(path) / np.sqrt(N)
print(stderr)
#c_path = np.exp(-r * T) * np.maximum(S_T - K, 0)
#p_path = np.exp(-r * T) * np.maximum(K - S_T, 0)




3.6856809616115433
0.019190778117340847


In [49]:
import matplotlib.pyplot as plt



array([ 49.25340789,  64.80272791,  31.10799   ,  43.56276208,
        50.94306384,  44.29982543,  73.25897928,  48.12543913,
        66.82694429,  34.55959075,  53.092153  ,  39.40545993,
        73.69741094,  62.01184318,  45.09128926,  53.35898203,
        47.49743712,  54.04608476,  44.26823075,  40.14624838,
        33.80004675,  60.4180674 ,  36.23744191,  35.55464316,
        59.06014246,  59.63558376,  55.18334655,  49.4865245 ,
        35.94221974,  39.92112782,  53.96991618,  26.95982745,
        35.56105215,  38.22108876,  64.43203289,  38.48529638,
        41.02260674,  58.90069941,  24.35656551, 100.07341809,
        24.26186627,  54.76769143,  82.40998342,  44.12658818,
        47.00425877,  66.93385235,  40.31707616,  34.93120393,
        36.48116584,  43.1576734 ,  43.47941869,  47.38035745,
        64.11913546,  46.31709056,  58.11985449,  74.30587136,
        90.60946786,  40.70792332,  29.86364286,  60.36513486,
        49.89396875,  42.17171392,  44.17596199,  23.65